In [1]:
!pip install numpy pandas torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/ab/6a/0debe1ec3c63b1fd7487ec7dd8fb1adf19898bef5a8dc151265d79ffd915/torch-2.1.0-cp310-none-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 MB 963.2 kB/s eta 0:00:0000:0100:02


In [6]:
import numpy as np
import pandas as pd
import torch

### Data loading

In [3]:
df = pd.read_csv("data_banknote_authentication.txt", header=None)
df

,0,1,2,3,4
0,3.62160,8.66610,-2.8073,-0.44699,0
1,4.54590,8.16740,-2.4586,-1.46210,0
2,3.86600,-2.63830,1.9242,0.10645,0
3,3.45660,9.52280,-4.0112,-3.59440,0
4,0.32924,-4.45520,4.5718,-0.98880,0
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1


In [4]:
X_features = df[[0, 1, 2, 3]].values
y_targets = df[4].values

In [5]:
np.bincount(y_targets)

array([762, 610])

### DataLoader

In [7]:
from torch.utils.data import Dataset, DataLoader

class BanknoteDataset(Dataset):

    def __init__(self, X, y):

        self._features = torch.tensor(X, dtype=torch.float32)
        self._labels = torch.tensor(y, dtype=torch.float32)

    def __getitem__(self, index):
        x = self._features[index]
        y = self._labels[index]

        return x, y

    def __len__(self):
        return self._labels.shape[0]


In [8]:
train_size = int(X_features.shape[0] * 0.8)
test_size = X_features.shape[0] - train_size

### Split data to train and test

In [9]:
dataset = BanknoteDataset(X_features, y_targets)

torch.manual_seed(1)
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(
    dataset=train_set,
    batch_size=10,
    shuffle=True,
)

val_loader = DataLoader(
    dataset=val_set,
    batch_size=10,
    shuffle=False,
)

In [10]:
train_mean = torch.zeros(X_features.shape[1])

for x, y in train_loader:
    train_mean += x.sum(dim=0)

train_mean /= len(train_set)

train_std = torch.zeros(X_features.shape[1])
for x, y in train_loader:
    train_std += ((x - train_mean) ** 2).sum(dim=0)

train_std = torch.sqrt(train_std / (len(train_set) - 1))

In [11]:
print("Feature means:", train_mean)
print("Feature std. devs:", train_std)

Feature means: tensor([ 0.3854,  1.8680,  1.4923, -1.1999])
Feature std. devs: tensor([2.8575, 5.9216, 4.3869, 2.1041])


In [12]:
def standardize(df, mean, std):
    for example in df:
        example = (example - mean) / std
    return df

### Implementing Logistic Regression Model

In [13]:
class LogisticRegressionModel(torch.nn.Module):

    def __init__(self, num_features):

        super().__init__()
        self._linear = torch.nn.Linear(num_features, 1)


    def forward(self, X):
        logits = self._linear(X)
        probas = torch.sigmoid(logits)
        return probas


### Model evaluation

In [14]:
def compute_accuracy(model, dataloader):

    model = model.eval()

    correct = 0.0
    total_examples = 0

    for idx, (features, class_labels) in enumerate(dataloader):

        with torch.no_grad():
            probas = model(features)

        pred = torch.where(probas > 0.5, 1, 0)
        lab = class_labels.view(pred.shape).to(pred.dtype)

        compare = lab == pred
        correct += torch.sum(compare)
        total_examples += len(compare)

    return correct / total_examples

### Train model

In [15]:
import torch.nn.functional as F

num_epochs_stop = 30
learning_rate_stop = 3

def find_optimal_hyperparameters():

    torch.manual_seed(1)
    model = LogisticRegressionModel(num_features=4)

    stop_learning = False
    best_lr = 0
    best_num_epochs = 0
    for epochs_count in range(1, num_epochs_stop):
        if stop_learning:
            break
        num_epochs = epochs_count
        curr_lr = 0.1
        while curr_lr < learning_rate_stop:
            if stop_learning:
                break
            optimizer = torch.optim.SGD(model.parameters(), lr=curr_lr)
            for epoch in range(num_epochs):
                if stop_learning:
                    break
                model = model.train()
                for batch_idx, (features, class_labels) in enumerate(train_loader):

                    features = standardize(features, train_mean, train_std)
                    probas = model(features)

                    loss = F.binary_cross_entropy(probas, class_labels.view(probas.shape))

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    ### LOGGING
                    if not batch_idx % 20: # log every 20th batch
                        print(f'Epoch: {epoch+1:03d}/{num_epochs:03d}'
                               f' | Batch {batch_idx:03d}/{len(train_loader):03d}'
                               f' | Loss: {loss:.2f}')
                    train_acc = compute_accuracy(model, train_loader)
                    val_acc = compute_accuracy(model, val_loader)
                    if train_acc * 100 > 98 and val_acc * 100 > 98:
                        best_lr = curr_lr
                        best_num_epochs = epochs_count
                        break

            curr_lr += 0.1

        return {"best_lr": best_lr, "best_num_epochs": best_num_epochs, "model": model}


In [16]:
res = find_optimal_hyperparameters()

Epoch: 001/001 | Batch 000/110 | Loss: 1.30
Epoch: 001/001 | Batch 020/110 | Loss: 0.17
Epoch: 001/001 | Batch 040/110 | Loss: 0.29
Epoch: 001/001 | Batch 060/110 | Loss: 0.07
Epoch: 001/001 | Batch 080/110 | Loss: 0.04
Epoch: 001/001 | Batch 100/110 | Loss: 0.05
Epoch: 001/001 | Batch 000/110 | Loss: 0.13
Epoch: 001/001 | Batch 020/110 | Loss: 0.23
Epoch: 001/001 | Batch 040/110 | Loss: 0.04
Epoch: 001/001 | Batch 060/110 | Loss: 0.04
Epoch: 001/001 | Batch 000/110 | Loss: 0.04
Epoch: 001/001 | Batch 000/110 | Loss: 0.02
Epoch: 001/001 | Batch 000/110 | Loss: 0.11
Epoch: 001/001 | Batch 000/110 | Loss: 0.01
Epoch: 001/001 | Batch 000/110 | Loss: 0.07
Epoch: 001/001 | Batch 000/110 | Loss: 0.01
Epoch: 001/001 | Batch 020/110 | Loss: 0.01
Epoch: 001/001 | Batch 000/110 | Loss: 0.08
Epoch: 001/001 | Batch 000/110 | Loss: 0.03
Epoch: 001/001 | Batch 000/110 | Loss: 0.03
Epoch: 001/001 | Batch 000/110 | Loss: 0.02
Epoch: 001/001 | Batch 000/110 | Loss: 0.02
Epoch: 001/001 | Batch 000/110 |

In [17]:
print(res)

{'best_lr': 2.9000000000000012, 'best_num_epochs': 1, 'model': LogisticRegressionModel(
  (_linear): Linear(in_features=4, out_features=1, bias=True)
)}


In [19]:
train_acc = compute_accuracy(res["model"], train_loader)
print(f"Train Accuracy: {train_acc*100:.2f}%")

val_acc = compute_accuracy(res["model"], val_loader)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

Train Accuracy: 98.27%
Validation Accuracy: 98.91%
